In [62]:
from compas.datastructures import Mesh
import compas.datastructures.mesh.subdivision as sd
from compas.geometry import Circle, Polyline, Line, closest_point_in_cloud
import ipyvolume as ipv
from utilities import draw_compas_mesh
from shapes import *
from compas.datastructures import meshes_join
from compas.geometry import distance_point_point

In [63]:
origin = (0,0,0)
normal = (0,0,1)
plane = (origin, normal)

r_axis_1 = 5
r_axis_2 = r_axis_1*1.1
r_pipe = .5

circle_1 = Circle(plane, r_axis_1)
circle_2 = Circle(plane, r_axis_2)

cylinder_1 = Cylinder(circle_1, 1)
cylinder_2 = Cylinder(circle_2, 1)

#torus = Torus(plane, r_axis, r_pipe)

vs, fs = cylinder_1.to_vertices_and_faces(u = 20, v = 3)
mesh_1 = Mesh.from_vertices_and_faces(vs, fs)

vs, fs = cylinder_2.to_vertices_and_faces(u = 20, v = 3)
mesh_2 = Mesh.from_vertices_and_faces(vs, fs)

#subd = sd.mesh_subdivide_catmullclark(mesh, k=1)

#draw_compas_mesh(subd)

In [98]:
# get lines from edges
# create new lines from closest pt other mesh
# mesh from lines

face_coords_list = []

    
connection_lines = []
existing_edge_lines = []

for m in [mesh_1, mesh_2]:
    for fkey in m.faces():
        face_coords = m.face_coordinates(fkey, axes='xyz')
        for i in range(len(face_coords)-1):
            line = [*face_coords[i]], [*face_coords[i+1]]
            existing_edge_lines.append(line)
           
for fkey in mesh_1.faces():
    face_coords = mesh_1.face_coordinates(fkey, axes='xyz')
    face_coords_list.append(face_coords)
    
pt_cloud_from_other_mesh = []

for vkey in mesh_2.vertices():
    v_coord = mesh_2.vertex_coordinates(vkey)
    pt_cloud_from_other_mesh.append(v_coord)
    
for face in face_coords_list:
    for vertex in face:
        _, friend, _ = closest_point_in_cloud(vertex, pt_cloud_from_other_mesh)
        line = [*vertex], [*friend]
        connection_lines.append(line)
        
print(connection_lines)
   
bunch_of_lines = connection_lines + existing_edge_lines

#print(bunch_of_lines)

joined_mesh = Mesh.from_lines(bunch_of_lines)

draw_compas_mesh(joined_mesh)

[([3.5355339059327373, 3.5355339059327373, 0.4999999999999999], [3.889087296526011, 3.889087296526011, 0.4999999999999999]), ([3.5355339059327373, 3.5355339059327373, -0.4999999999999999], [3.889087296526011, 3.889087296526011, -0.4999999999999999]), ([2.2699524986977337, 4.4550326209418385, -0.4999999999999999], [2.4969477485675067, 4.900535883036023, -0.4999999999999999]), ([2.2699524986977337, 4.4550326209418385, 0.4999999999999999], [2.4969477485675067, 4.900535883036023, 0.4999999999999999]), ([2.2699524986977337, 4.4550326209418385, 0.4999999999999999], [2.4969477485675067, 4.900535883036023, 0.4999999999999999]), ([2.2699524986977337, 4.4550326209418385, -0.4999999999999999], [2.4969477485675067, 4.900535883036023, -0.4999999999999999]), ([0.7821723252011541, 4.938441702975688, -0.4999999999999999], [0.8603895577212701, 5.432285873273257, -0.4999999999999999]), ([0.7821723252011541, 4.938441702975688, 0.4999999999999999], [0.8603895577212701, 5.432285873273257, 0.499999999999999

ZeroDivisionError: float division by zero

In [53]:
joined = meshes_join([mesh_1, mesh_2])
joined.summary()
draw_compas_mesh([mesh_1, mesh_2])


Mesh summary

- name: Mesh
- vertices: 80
- edges: 120
- faces: 44
- vertex degree: 3/3
- face degree: 4/20




AttributeError: 'list' object has no attribute 'to_vertices_and_faces'

In [39]:
area_list = []


for fkey in mesh.faces():
    area = mesh.face_area(fkey)
    #print(area)
    area_list.append(area)
    
#max_area = max(area_list)
#print(max_area)

average_area = sum(area_list) / float(len(area_list))

mesh2 = mesh.copy()

for fkey in mesh.faces():
    if mesh2.face_area(fkey) > average_area:
        print(mesh.face_area(fkey))
        mesh2.delete_face(fkey)
        
draw_compas_mesh(mesh2)

        

77.25424859373683
77.25424859373682


Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [28]:
distances = []

for v in mesh.vertices():
    v_coord = mesh.vertex_coordinates(v)
    distance_to_mid = distance_point_point(origin, v_coord)
    distances.append(distance_to_mid)

print(distances)

[7.071067811865475, 7.071067811865475, 7.071067811865475, 7.071067811865475, 7.071067811865475, 7.071067811865475, 7.071067811865475, 7.071067811865475, 7.0710678118654755, 7.0710678118654755, 7.071067811865475, 7.071067811865475]


In [3]:
distance_outer_ring = round(max(distances), 1)
distance_inner_ring = round(min(distances), 1)

print(distance_inner_ring)
print(distance_outer_ring)

4.5
5.5


In [4]:
point_list = []

for v in mesh.vertices():
    v_coord = mesh.vertex_coordinates(v)
    point_list.append(v_coord)
    distance_to_mid = distance_point_point(origin, v_coord)
     
    if distance_to_mid == distance_outer_ring:
        mesh.set_vertex_attribute(v, 'location', 'outer')
    else:
        mesh.set_vertex_attribute(v, 'location', 'inner')

        
new_mesh = Mesh()

for pt in point_list:
    new_mesh.add_vertex(x=pt[0], y=pt[1], z=pt[0])
        
print(point_list)
new_mesh = Mesh.from_points(point_list)
draw_compas_mesh(new_mesh)



[[3.889087296526011, 3.889087296526011, 0.0], [3.5355339059327373, 3.5355339059327373, 0.4999999999999999], [3.181980515339464, 3.181980515339464, 6.123233995736765e-17], [3.5355339059327373, 3.5355339059327373, -0.4999999999999999], [1.4235047480638647, 5.312592044589875, 0.0], [1.2940952255126041, 4.829629131445341, 0.4999999999999999], [1.1646857029613438, 4.346666218300807, 6.123233995736765e-17], [1.2940952255126041, 4.829629131445341, -0.4999999999999999], [-1.4235047480638636, 5.312592044589875, 0.0], [-1.2940952255126033, 4.829629131445341, 0.4999999999999999], [-1.1646857029613427, 4.346666218300807, 6.123233995736765e-17], [-1.2940952255126033, 4.829629131445341, -0.4999999999999999], [-3.8890872965260104, 3.8890872965260113, 0.0], [-3.5355339059327373, 3.5355339059327373, 0.4999999999999999], [-3.181980515339464, 3.181980515339464, 6.123233995736765e-17], [-3.5355339059327373, 3.5355339059327373, -0.4999999999999999], [-5.312592044589874, 1.4235047480638654, 0.0], [-4.829629

TypeError: 'bool' object is not iterable